In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from gensim import corpora, models, similarities
import pickle
import pandas as pd
!pip install wordcloud
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import urllib
import requests

     |████████████████████████████████| 368kB 2.4MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
df = pickle.load(open('df_preprocessed.pickle', 'rb'))
df.reset_index(inplace=True)

In [3]:
df_unprocessed = pickle.load(open('df', 'rb'))

In [4]:
df['Unprocessed Title'] = df_unprocessed['Title'].astype(str)

In [5]:
df.head()

,Date,Title,Body,Unprocessed Title
0,2020-04-16,"[sipping, coronavirus, domain, firehose]","[expert, poring, thousand, new, coronavirusthe...",Sipping from the Coronavirus Domain Firehose —...
1,2020-04-15,"[covid19, united, cybersecurity, expert, unity...","[coronavirus, prompted, thousand, information,...","COVID-19 Has United Cybersecurity Experts, But..."
2,2020-04-14,"[microsoft, patch, tuesday, april, 2020, edition]","[microsoft, today, released, update, fix, 113,...","Microsoft Patch Tuesday, April 2020 Edition — ..."
3,2020-04-10,"[new, irs, site, could, make, easy, thief, int...","[u, federal, government, process, sending, eco...",New IRS Site Could Make it Easy for Thieves to...
4,2020-04-07,"[microsoft, buy, corpcom, bad, guy, cant]","[february, told, story, private, citizen, auct...",Microsoft Buys Corp.com So Bad Guys Can’t — K...


In [6]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))

In [7]:
tfidf_matrix = vectorizer.fit_transform(df['Body'].apply(', '.join))

In [38]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [56]:
def get_recommendations(title, cosine_sim):
    #Get the index 
    indices = pd.Series(df.index, index=df['Unprocessed Title']).drop_duplicates()
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:9]
    article_indices = [i[0] for i in sim_scores]
    return df['Unprocessed Title'].iloc[article_indices]
    

In [65]:
print(df['Unprocessed Title'][1])

COVID-19 Has United Cybersecurity Experts, But Will That Unity Survive the Pandemic? —  Krebs on Security


In [67]:
test_title_1 = 'ATM Skimmers, Part II —  Krebs on Security'
test_title_2 = 'Microsoft Buys Corp.com So Bad Guys Can’t —  Krebs on Security'
get_recommendations(test_title_2, cosine_sim)

31      Dangerous Domain Corp.com Goes Up for Sale —  ...
203     That Domain You Forgot to Renew? Yeah, it’s No...
175     Bomb Threat, Sextortion Spammers Abused Weakne...
38      Does Your Domain Have a Registry Lock? —  Kreb...
0       Sipping from the Coronavirus Domain Firehose —...
1348    Microsoft to Botmasters: Abandon Your Inboxes ...
262     Bad .Men at .Work. Please Don’t .Click —  Kreb...
1301    DoItQuick: Fast Domains for Dirty Deeds —  Kre...
Name: Unprocessed Title, dtype: object

In [63]:
df['Date'][31]

Timestamp('2020-02-08 00:00:00')

In [64]:
df['Date'][4]

Timestamp('2020-04-07 00:00:00')